<a href="https://colab.research.google.com/github/tarang1998/ML-AI-DL/blob/main/gen-ai/langchain/IntroToLangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [3]:
import os
import openai
import getpass

In [2]:


os.environ['OPENAI_API_KEY'] = getpass.getpass(prompt='Enter OpenAI API key:')



Enter OpenAI API key:··········


In [12]:
os.environ["HF_TOKEN"] = getpass.getpass(prompt='Enter HuggingFace API key:')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.environ["HF_TOKEN"]

Enter HuggingFace API key:··········


In [3]:
!pip install openai

In [4]:
# Using LangChain with Open AI

from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)

<ipython-input-4-61402f2475db>:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9)


In [5]:
# Example 1
text="What would be a good company name for a company that makes colorful socks?"

In [6]:
print(llm.predict(text))

<ipython-input-6-652b389f66d4>:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm.predict(text))




"Rainbow Feet Co." or "Vibrant Soles Co."


In [7]:
print(llm(text))

<ipython-input-7-605043303f29>:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm(text))



"Spectrum Socks Co." 


In [8]:
print(llm.invoke(text))



"Rainbow Socks Co." 


In [9]:
# Example 2
text2 = "I want to open a restaurant for Chinese food. Suggest a fency name for this."

In [10]:
name = llm.predict(text2)
print(name)



"The Jade Dragon Restaurant"


In [11]:
# Using LangChain with Hugging Face

!pip install huggingface_hub

In [12]:
from huggingface_hub import InferenceClient


In [16]:
# Initialize the client with the model repo ID
client = InferenceClient(model="google/flan-t5-large")

# Make a text generation request
prompt1 = "I want to open a restaurant for Indian food. Suggest a fancy name for this."
response = client.text_generation(prompt1, max_new_tokens=64, temperature= 0)

# Print the response
print(response)

Indian restaurant


In [17]:
prompt2 = "translate English to German: How old are you?"
response = client.text_generation(prompt2, max_new_tokens=64, temperature= 0)
print(response)

Wie alte sind Sie?


In [19]:
# Using Prompt templates
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)
p = prompt_template_name.format(cuisine="indian")
print(p)

I want to open a restaurant for indian food. Suggest a fancy name for this.


In [20]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks'

In [24]:
# Chains : Combine LLMs and Prompts in multi-step workflows

## LLM Chain

from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")
prompt.format(product="colorful socks")

from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
response= chain.invoke("colorful socks")
print(response)

{'product': 'colorful socks', 'text': '\n\nRainbow Socks Co. or ChromaSock Co.'}


In [25]:
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

chain = LLMChain(llm=llm, prompt=prompt_template_name)
response=chain.invoke("Mexican")
print(response)

{'cuisine': 'Mexican', 'text': '\n\n"El Sabor de México" (The Flavor of Mexico)'}


In [26]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
response=chain.invoke("Mexican")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.
{'cuisine': 'Mexican', 'text': '\n\n"El Sabor Exquisito"'}


In [27]:
# Simple Sequential Chain

llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [28]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("indian")
print(content)



1. Spicy Chicken Tikka Masala
2. Lamb Vindaloo
3. Vegetable Samosas
4. Tandoori Shrimp
5. Palak Paneer (spinach and cottage cheese curry)
6. Aloo Gobi (potatoes and cauliflower curry)
7. Chana Masala (chickpea curry)
8. Butter Chicken
9. Biryani (spiced rice dish with choice of protein)
10. Naan bread
11. Mango Lassi (yogurt-based drink)
12. Tandoori Chicken skewers
13. Saag Aloo (spinach and potato curry)
14. Malai Kofta (vegetable dumplings in a creamy sauce)
15. Chicken or Vegetable Korma (mild and creamy curry)


In [30]:
# Sequential Chain

from langchain.chains import SequentialChain

llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

print(chain({"cuisine": "indian"}))


<ipython-input-30-96ff033a781c>:27: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain({"cuisine": "indian"}))


{'cuisine': 'indian', 'restaurant_name': '\n\n"Spice Oasis"', 'menu_items': '\n\n1. Spicy Chicken Tikka Masala\n2. Vegetable Samosas\n3. Lamb Vindaloo\n4. Tandoori Shrimp\n5. Aloo Gobi (spiced cauliflower and potatoes)\n6. Chana Masala (spicy chickpea curry)\n7. Chicken Korma\n8. Palak Paneer (spinach and cheese curry)\n9. Dum Biryani (spicy rice dish with choice of protein)\n10. Naan bread with garlic and chili flakes\n11. Mango Lassi (spiced yogurt drink)\n12. Beef Madras\n13. Vegetable Biryani\n14. Tandoori Tofu Tikka\n15. Spicy Vada Pav (Indian spiced potato burger)\n16. Coconut Curry Shrimp\n17. Saag Chicken (spicy chicken and spinach curry)\n18. Spicy Mango Chutney\n19. Malai Kofta (spicy vegetable dumplings in creamy sauce)\n20. Gulab Jamun (spicy sweet Indian dessert).'}


In [31]:
# Agents and Tools

!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=4457745a2700a305808e23d04f174a08f676c0f6cfeab92a028242a0341134ca
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [32]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

In [33]:
llm = OpenAI(temperature=0)

In [38]:
# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia",], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)



agent.run("What was the GDP of US in 2024?")



> Entering new AgentExecutor chain...
 I should always think about what to do
Action: wikipedia
Action Input: GDP of US in 2024
Observation: Page: List of U.S. states and territories by GDP
Summary: This is a list of U.S. states and territories by gross domestic product (GDP). This article presents the 50 U.S. states and the District of Columbia and their nominal GDP at current prices.
The data source for the list is the Bureau of Economic Analysis (BEA) in 2024. The BEA defined GDP by state as "the sum of value added from all industries in the state."
Nominal GDP does not take into account differences in the cost of living in different countries, and the results can vary greatly from one year to another based on fluctuations in the exchange rates of the country's currency. Such fluctuations may change a country's ranking from one year to the next, even though they often make little or no difference in the standard of living of its population.
Overall, in the calendar year 2024, the 

'The GDP of the US in 2024 was $29.017 trillion.'

In [39]:
# Memory
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)


from langchain.chains import LLMChain

chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



"Casa Delicioso"


In [40]:
name = chain.run("Indian")
print(name)


"Fusion Spice House" 


In [41]:
type(chain.memory)

NoneType

In [42]:
# Conversation Buffer Memory

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

<ipython-input-42-c6cf17605daf>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()




"La Cantina del Sol"


In [43]:
name = chain.run("Arabic")
print(name)



"Sahara Palace"


In [44]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"La Cantina del Sol"
Human: Arabic
AI: 

"Sahara Palace"


In [45]:
# Conversation Chain

from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


<ipython-input-45-e9279c385800>:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  convo = ConversationChain(llm=OpenAI(temperature=0.7))


In [46]:
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was won by the West Indies team in 1975. It was held in England and consisted of eight teams, with India being the runner-up. The West Indies team was led by Clive Lloyd and had players like Viv Richards and Andy Roberts. They defeated Australia in the final by 17 runs.'

In [47]:
convo.run("How much is 5+5?")

'  5+5 is equal to 10.'

In [48]:
convo.run("Who was the captain of the winning team?")

' The captain of the winning team, the West Indies, was Clive Lloyd. He was a left-handed batsman and a medium-paced bowler. He was known for his strong leadership skills and led the West Indies team to two consecutive World Cup victories in 1975 and 1979. '

In [49]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies team in 1975. It was held in England and consisted of eight teams, with India being the runner-up. The West Indies team was led by Clive Lloyd and had players like Viv Richards and Andy Roberts. They defeated Australia in the final by 17 runs.
Human: How much is 5+5?
AI:   5+5 is equal to 10.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team, the West Indies, was Clive Lloyd. He was a left-handed batsman and a medium-paced bowler. He was known for his strong leadership skills and led the West Indies team to two consecutive World Cup victories in 1975 and 1979. 


In [50]:
# Conversation Buffer Window Memory

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

<ipython-input-50-f16f53a72db3>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=3)


" The first cricket world cup was won by the West Indies in 1975. The final match was played between the West Indies and Australia at Lord's Cricket Ground in London. The West Indies won by 17 runs. The captain of the West Indies team was Clive Lloyd and the man of the match was Viv Richards. This was a historic moment for the West Indies as it was the first time they had won the cricket world cup."

In [51]:
convo.run("How much is 5+5?")

'  5+5 is equal to 10.'

In [52]:
convo.run("Who was the captain of the winning team?")

' The captain of the winning team was Clive Lloyd. He was a legendary cricketer from the West Indies and led his team to victory in the first cricket world cup in 1975. He was also the captain of the West Indies team when they won the second cricket world cup in 1979. He is considered one of the greatest captains in the history of cricket.'

In [53]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies in 1975. The final match was played between the West Indies and Australia at Lord's Cricket Ground in London. The West Indies won by 17 runs. The captain of the West Indies team was Clive Lloyd and the man of the match was Viv Richards. This was a historic moment for the West Indies as it was the first time they had won the cricket world cup.
Human: How much is 5+5?
AI:   5+5 is equal to 10.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team was Clive Lloyd. He was a legendary cricketer from the West Indies and led his team to victory in the first cricket world cup in 1975. He was also the captain of the West Indies team when they won the second cricket world cup in 1979. He is considered one of the greatest captains in the history of cricket.


In [54]:
convo.run("Who was the captain of the winning team?")

' The captain of the winning team was Clive Lloyd.'

In [55]:
print(convo.memory.buffer)

Human: How much is 5+5?
AI:   5+5 is equal to 10.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team was Clive Lloyd. He was a legendary cricketer from the West Indies and led his team to victory in the first cricket world cup in 1975. He was also the captain of the West Indies team when they won the second cricket world cup in 1979. He is considered one of the greatest captains in the history of cricket.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team was Clive Lloyd.


In [1]:
# Loading PDF documents

!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 7.7 MB/s eta 0:00:00


In [7]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./content/TarangResume.pdf")
pages = loader.load()

In [8]:
pages

[Document(metadata={'producer': 'Skia/PDF m134 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Resume - Tarang Nair', 'source': './content/TarangResume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='TARANG  NAIR  Arlington,  Virginia  |  571-385-5625  |  tarang98@umd.edu |  linkedin.com/in/tarangnair1998 |  github.com/tarang1998 |  tarangnair.com  \nEDUCATION  University  Of  Maryland,  College  Park                    Maryland,  United  States  Masters  Of  Engineering  in  Software  Engineering                                      Expected  2025  \nVidyalankar\n \nInstitute\n \nOf\n \nTechnology,\n \nMumbai\n \nUniversity\n \n                    \n \n \n \n                                        \nMumbai,\n \nIndia\n Bachelors  Of  Engineering  in  Computer  Engineering                                                    July  2016  -  July  2020   \nWORK  EXPERIENCE  FINRA                            Rockville,  MD,  USA  (Hybrid)     \nS

In [4]:
# Agents accessing multiple data frames

!pip install langchain langchain_experimental
!pip install watermark
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.5 MB/s eta 0:00:00


In [5]:
import os
import warnings
warnings.filterwarnings("ignore")

In [6]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.llms import OpenAI

In [7]:
import pandas as pd

In [8]:
url = "https://raw.githubusercontent.com/adamerose/datasets/master/titanic.csv"
df=pd.read_csv(url)
print(df.shape)
df.head()

(891, 15)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [9]:
# Agent interacting with a single dataframe

llm=OpenAI()

<ipython-input-9-bc21021cf673>:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI()


In [10]:
agent=create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code=True)

In [11]:
agent.run("How many rows are there")

<ipython-input-11-a556688d9c7e>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("How many rows are there")




> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)891891 rows is a lot, but I'm not sure if that's the final answer
Final Answer: 891

> Finished chain.


'891'

In [12]:
agent.run("How many people have more than 23 age")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe for people with age greater than 23 and then count the number of rows.
Action: python_repl_ast
Action Input: df[df['age'] > 23]     survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
6           0       1    male  54.0      0      0  51.8625        S   First   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
884         0       3    male  25.0      0      0   7.0500        S   Third   
885         0       3  female  39.0      0      5  29.1250        Q   Third   
886         0       2    male  27.0      0      0  13.0000        S  Second   
889

'468'

In [13]:
# Agents interacting with multiple data frames

df1=df.copy()
df1["age"] = df1["age"].fillna(df1["age"].mean())

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [14]:
agent=create_pandas_dataframe_agent(llm, [df, df1], verbose=True,allow_dangerous_code=True)
agent.run("How many rows in the age column are different")



> Entering new AgentExecutor chain...
Thought: We can compare the two dataframes and count the rows where the age column is different.
Action: python_repl_ast
Action Input: df1['age'] != df2['age']0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888     True
889    False
890    False
Name: age, Length: 891, dtype: boolNow we need to count the number of rows with the value True.
Action: python_repl_ast
Action Input: (df1['age'] != df2['age']).sum()177177 rows have different values in the age column.
Final Answer: 177 

> Finished chain.


'177'

In [15]:
df2=df1.copy()
df2["Age_Multiplied"]=df1["age"] * 2
df2.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,Age_Multiplied
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,44.0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,76.0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,52.0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,70.0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,70.0


In [16]:
agent=create_pandas_dataframe_agent(llm, [df,df1, df2], verbose=True, allow_dangerous_code=True)

In [1]:
# LangChain with hugging face

!pip install langchain langchain_community
#For API Calls
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install  bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 766.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [23]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from huggingface_hub import InferenceClient


In [24]:
# Interacting with hugging face API for text generation


# Initialize the Hugging Face inference client
client = InferenceClient(model="google/flan-t5-large")

# Define a prompt template
prompt_template = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?"
)

# Function to generate text using InferenceClient
def generate_name(product):
    prompt = prompt_template.format(product=product)
    response = client.text_generation(prompt, max_new_tokens=64)
    return response.strip()  # Clean the response

# Run inference
print(generate_name("colorful socks"))


sock mania


In [28]:

# Initialize the Hugging Face inference client
client = InferenceClient(model="google/flan-t5-large")

# Define a prompt template
prompt_template = PromptTemplate(
    input_variables=["name"],
    template="Can you tell me about famous footballer {name}"
)

# Function to generate text using InferenceClient
def generate_name(name):
    prompt = prompt_template.format(name=name)
    response = client.text_generation(prompt, max_new_tokens=64)
    return response.strip()  # Clean the response

# Run inference
print(generate_name("Messi"))



Messi is a footballer who plays for Argentina.


In [30]:
# Download the model locally - Create pipelines

from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# Load model and tokenizer
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create a text generation pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=128)

# Use HuggingFacePipeline with the pipeline object
llm = HuggingFacePipeline(pipeline=pipe)

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?"
)

# Create the LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain
print(chain.run("colorful socks"))

Device set to use cuda:0


sock mania


In [31]:
prompt = PromptTemplate(
    input_variables=["name"],
    template="Can you tell me about famous footballer {name}"
)

chain = LLMChain(llm=llm, prompt=prompt)

chain.run("Messi")

'Messi is a footballer who plays for Argentina.'